In [1]:
import pandas as pd

import scipy as sc
from scipy.io import loadmat

import numpy as np

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel

np.random.seed(123)

# Read Data

In [5]:
# splits
# images
# distances matrix
DATA_DIR = "../data"

# DATASET1: 17 Category Flowers
# 'val1', 'val2', 'val3', 'trn1', 'trn2', 'trn3', 'tst3', 'tst2', 'tst1'
'''splits = loadmat("%s/cat_flower/datasplits.mat" %DATA_DIR)

# 'D_siftbdy', 'D_siftint', 'D_hsv', 'D_hog',
distances = loadmat("%s/cat_flower/distancematrices17itfeat08.mat" %DATA_DIR)

imlist = loadmat("%s/cat_flower/trimaps/imlist.mat" %DATA_DIR)
'''

# DATASET2: 102 Category Flowers
# 'val1', 'val2', 'val3', 'trn1', 'trn2', 'trn3', 'tst3', 'tst2', 'tst1'
ds2_splits = loadmat("%s/102_cat_flower/setid.mat" %DATA_DIR)

# 'D_siftbdy', 'D_siftint', 'D_hsv', 'D_hog',
ds2_distances = loadmat("%s/102_cat_flower/distancematrices102.mat" %DATA_DIR)

ds2_labels = loadmat("%s/102_cat_flower/imagelabels.mat" %DATA_DIR)

# Prepare

In [4]:
'''KERNELS = {
    'gaussian': get_gaussian_kernel,
    'poly': get_ploynomial_kernel,
    'dist': get_distances_to_kernel
}'''

def get_gaussian_kernel(X, Y=None, width=.2):
    if Y is None:
        Y = X
        
    return rbf_kernel(X, gamma=1.0/width)

def get_ploynomial_kernel(X, Y=None, deg=2):
    if Y is None:
        Y = X
        
    return polynomial_kernel(X, Y, degree=deg)

def get_distances_to_kernel(X):
    return (-X/X.mean())

def get_kernels(X, Y=None, poly=False, gauss=False, distk=False):
    kernels = []
    
    if Y is None:
        Y = X
    
    if poly:
        degrees = [1,2,3]
        
        for deg in degrees:
            kernels.append(get_ploynomial_kernel(X, Y, deg=deg))
            
            
    if gauss:
        widths = map(lambda x: 2**x, np.arange(-3,6))
        
        for w in widths:
            kernels.append(get_gaussian_kernel(X, Y, width=w))
            
            
    if distk:
        kernels.append(get_distances_to_kernel(X))
        
    return arr(kernels)
    
    

In [5]:
# Helper Functions
log = np.log
gamma = sc.special.gamma
digamma = sc.special.digamma
det = np.linalg.det
diag = np.diag
outer = np.outer
tr = np.trace
dot =  np.dot
concat = np.concatenate
normal = np.random.normal
arr = np.array
cstack = np.column_stack
sqrt = np.sqrt
inv = np.linalg.inv
norm = sc.stats.norm
append = np.append

# Dummy

In [6]:
#x1 = np.random.normal(0,1,100)
x1 = np.random.multivariate_normal(mean=[0,0], cov=[[.1,0],[0,.1]], size=100)
y1 = np.repeat(1,100)
#x2 = np.random.normal(50,1,100)
x2 = np.random.multivariate_normal(mean=[2,2], cov=[[.1,0],[0,.1]], size=100)
y2 = np.repeat(-1,100)

N = 200
x = np.concatenate((x1,x2))
y = np.concatenate((y1,y2))

#K = arr([get_ploynomial_kernel(x.reshape((-1,1)), i+1) for i in range(3)])
#K = arr([get_ploynomial_kernel(x, i+1) for i in range(3)])
K = get_kernels(x, poly=True)
print K[0].shape

P = 3

(200, 200)


In [8]:
x.shape

(200, 2)

# BEMKL

<font size=4> $\lambda_i \sim Gamma(\alpha_\lambda,\beta_\lambda)$ <br>
    $E\_a2_i = E(a_i^2) = Var(a_i) + \mu_{a_i}^2$ <br>
    $Corr\_a = E(aa^T) = \Sigma_a + \mu_{a}\mu_{a}^T$ <br>
    $corr\_g_i = E(g_ig_i^T) = \Sigma_{g_i} + \mu_{g_i}\mu_{g_i}^T$ <br>
    $corr_G$ <br>
     <br>
</font>

### don't initialize to zero, use small non-zero values

In [29]:
np.random.seed(123)

MAX_ITER = 500

# Hyper Parameters (prior)
sparse = False

alpha_lambda, beta_lambda = 1.0,1.0
alpha_gamma, beta_gamma = 1.0,1.0
alpha_omega, beta_omega = 1.0,1.0

if sparse:
    alpha_omega, beta_omega = 10.0**-10, 10.0**10
    
v = 1

'''rv_lambda = np.random.gamma(alpha_lambda, beta_lambda, N) 
rv_a = np.array([np.random.normal(0, 1.0/rv_lambda[i], 1) for i in range(N)])

rv_gamma = np.random.gamma(alpha_gamma, beta_gamma, 1) 
rv_b = np.random.normal(0, 1.0/rv_gamma, 1)

rv_omega = np.random.gamma(alpha_omega, beta_omega, P) 
rv_e = np.array([np.random.normal(0, 1.0/rv_omega[i], 1) for i in range(P)])'''

# The conditionals parameters
p_alpha_lambda = np.repeat(alpha_lambda, N)
p_beta_lambda = np.repeat(beta_lambda, N)
p_alpha_gamma = alpha_gamma
p_beta_gamma = beta_gamma
p_alpha_omega = np.repeat(alpha_omega, P)
p_beta_omega = np.repeat(beta_omega, P)


mu_b = 0
#var_b = (alpha_gamma*beta_gamma)**-1
var_b = 1
#mu_e = np.repeat(0, P)
#mu_e = np.random.normal(size=P)
#mu_e = np.random.rand(P) + 1e-5
mu_e = np.ones(P)
mu_b_e = np.concatenate(([mu_b],mu_e))
#cov_e = diag(np.repeat((alpha_omega*beta_omega)**-1, P))
cov_e = np.eye(P)
cov_b_e = np.zeros((P+1,P+1))
cov_b_e[0,0] = var_b
cov_b_e[1:,1:] = cov_e
corr_e = cov_b_e[1:,1:] + outer(mu_b_e[1:], mu_b_e[1:])
print 'COV_b_e', cov_b_e

#mu_a = np.repeat(0, N)
mu_a = np.random.normal(0,1, size=N)
cov_a = diag(np.repeat((alpha_lambda*beta_lambda)**-1, N))
corr_a = cov_a + outer(mu_a, mu_a)

print 'cov_a',cov_a[:5,:5]


#mu_g = arr([dot(mu_a.T,K[j,...]) for j in range(P)]).squeeze()
#mu_g = np.random.normal(size=P*N).reshape((P,-1))
mu_g = (np.abs(np.random.normal(0,1,size=(P,N))) + v) * y

print 'mu_g', mu_g[:,:3], mu_g[:,-3:]

'''cov_g = arr([(outer(G[:,i],G[:,i])- outer(mu_g[:,i], mu_g[:,i])) 
                  for i in range(N)])
'''
# inter-column covariance: NxPxP
cov_g = np.zeros((N,P,P))
cov_g[:,:,:] = np.eye(P) 
corr_g = cov_g + arr([outer(mu_g[:,i], mu_g[:,i]) for i in range(N)])

# inter-row covariance: PxP
#mu_f = dot(mu_b_e[1:], mu_g) + mu_b
mu_f = (np.random.rand(N) + 1e-5) * y

print 'mu_f',mu_f[:3], mu_f[-3:]
#print 'cov_g',cov_g[:5,...]



#omega = np.random.gamma(alpha_omega, 1.0/beta_omega, P)
#e = concat([normal(0,omega[i],1) for i in range(P)])
#print 'e', e.shape

#mu_f = np.array([dot(e.T,G[:,i])+ rv_b for i in range(N)])
#mu_f = dot(e.T,mu_g)+ mu_b 


E_a2 = np.diag(cov_a) + mu_a**2 
E_b2 = alpha_gamma * beta_gamma + mu_b**2
E_e2 = np.diag(cov_b_e[1:,1:]) + mu_b_e[1:]**2
E_be = 0


thresh = 1.0 * 10**-4
ELBO_init = 0

K2 = np.sum([dot(K[i,...], K[i,...].T) for i in range(P)], axis=0)

COV_b_e [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
cov_a [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
mu_g [[1.70331012 1.59810533 3.2007021 ]
 [2.53409029 1.5299141  1.49097228]
 [2.14065624 1.78816556 1.26523396]] [[-1.24537941 -1.16842643 -1.24402694]
 [-2.39437511 -1.91268794 -2.27357024]
 [-2.37449688 -1.54706565 -1.00007553]]
mu_f [0.3186439  0.48906919 0.57221384] [-0.658099   -0.5932479  -0.22953345]


### Forms:
<font size=4> 
    $E(\alpha^T A \alpha) = tr(A\Sigma_{\alpha}) + \mu_{\alpha}^T A \mu_{alpha}$ <br>
    $Cov(g_i^m, g_i^n) = tr(A\Sigma_{a}) + \mu_{a}^T A \mu_{a}$ <br>
    where $A = K_{m,i}K_{n,i}^T$ <br>
    <br>
    $E(g^m, {g^n}^T) = tr(A\Sigma_{a}) + \mu_{a}^T A \mu_{a}$ <br>
    where $A = K_{m,i}K_{n,i}^T$
    <br>
</font>

### Tricks:
- for inverting the matrix add $\epsilon I$ to the matrix

In [30]:
it = 1
v = 1

while True: 
    # Update the parameters:
    ########################
    
    '''
    ############
    Update a
    ############
    '''
    p_alpha_lambda = np.repeat(alpha_lambda + 0.5, N)
    p_beta_lambda = (beta_lambda**-1 + E_a2/2.0)**-1
    E_lambda = p_alpha_lambda * p_beta_lambda
    
    
    cov_a = inv(diag(E_lambda) + K2)
    z = np.sum([dot(K[i,:,:], mu_g[i,:]) for i in range(P)], axis=0)
    mu_a = dot(cov_a, z)
    corr_a = cov_a + outer(mu_a, mu_a)
    E_a2 = diag(cov_a) + mu_a**2
    
    print 'Updated a...'
    print 'mu_a', mu_a[:5]
    print 'cov_a', cov_a[:5,:5]
    
    '''
    ############
    Update G
    ############
    '''
    #z = dot(cov_b_e[0,1:],inv(cov_b_e[1:,1:]))
    #E_be = mu_b_e[0] * mu_b_e[1:] + dot(corr_e, z) - dot(dot(mu_b_e[1:], z), mu_b_e[1:])    
    cov_g[:,:,:] = inv(np.eye(P) + corr_e)
    mu_g = cstack([dot(cov_g[i,:,:], (dot(K[:,i,:], mu_a) + mu_f[i]* mu_b_e[1:] 
                                      - E_be)) for i in range(N)])        
    corr_g = cov_g + arr([outer(mu_g[:,i], mu_g[:,i]) for i in range(N)])
    #cov_G = diag(cstack([diag(cov_g[i,...]) for i in range(N)]).sum(axis=1))
    #corr_G = cov_G + dot(mu_g,mu_g.T)
    '''corr_G = np.zeros((P,P))
    
    for i in range(P):
        for j in range(i, P):
            corr_G[i,j] = np.sum(cov_g[:,i,j] + mu_g[i,:]*mu_g[j,:])
            corr_G[j,i] = corr_G[i,j]'''
    corr_G = dot(mu_g,mu_g.T) + cov_g.sum(axis=0)  # exactly the same
    
    print 'Updated g...'
    print 'mu_g',mu_g[:,:3]
    print mu_g[:,-3:]
    print 'cov_g', cov_g[:1,...]
    print 'corr_G', corr_G
    
    '''
    ############
    Update e, b
    ############
    ''' 
    #E_b2 = cov_b_e[0,0] + mu_b_e[0]**2
    p_alpha_gamma = alpha_gamma + 0.5
    p_beta_gamma = (beta_gamma**-1 + E_b2/2.0)**-1
    
    p_alpha_omega = np.repeat(alpha_omega + 0.5, P)
    p_beta_omega = (beta_omega**-1 + E_e2/2.0)**-1
    
    E_gamma = p_alpha_gamma * p_beta_gamma
    E_omega = p_alpha_omega * p_beta_omega 
    

    cov_b_e[0,0] = E_gamma + N
    cov_b_e[1:,0] = mu_g.sum(axis=1)
    cov_b_e[0,1:] = cov_b_e[1:,0]
    cov_b_e[1:,1:] = diag(E_omega) + corr_G
    try:  cov_b_e = inv(cov_b_e)
    except: cov_b_e = inv(cov_b_e + 1e-5 * np.eye((P+1,P+1)))
        
    z = concat(([mu_f.sum()], dot(mu_g,mu_f).squeeze()))
    mu_b_e = dot(cov_b_e, z)
    
    E_be = mu_b_e[0] * mu_b_e[1:] + cov_b_e[0,1:]
    E_b2 = cov_b_e[0,0] + mu_b_e[0]**2
    E_e2 = diag(cov_b_e[1:,1:]) + mu_b_e[1:]**2
    corr_e = cov_b_e[1:,1:] + outer(mu_b_e[1:], mu_b_e[1:])
    
    print 'Updated e:'
    print 'mu_b_e', mu_b_e
    print 'cov_b_e', cov_b_e
    print 'corr_e', corr_e
    print 'E_be', E_be
    
    '''
    ############
    Update f
    ############
    '''
    LU = arr(map(lambda x: (-1e40, -v) if x==-1 else (v,1e40), y))
    eg = dot(mu_b_e[1:].T, mu_g)
    alpha_f = LU[:,0] - eg - mu_b_e[0]
    beta_f = LU[:,1] - eg - mu_b_e[0]
    pdf_alpha_f = norm.pdf(alpha_f)
    pdf_beta_f = norm.pdf(beta_f)
    Z = norm.cdf(beta_f) - norm.cdf(alpha_f)
    msk = (Z == 0.0)
    Z[msk] = 1.0
    mu_f = eg + mu_b_e[0] + (pdf_alpha_f - pdf_beta_f)/Z
    var_f = 1. + (alpha_f * pdf_alpha_f - beta_f * pdf_beta_f)/Z - (pdf_alpha_f - pdf_beta_f)**2/Z**2 
    E_f2 = var_f + mu_f**2
    
    print 'Updated f'
    print 'mu_f',mu_f[:3],mu_f[-3:]
    print 'var_f',var_f[:3]
    print 'E_f2', E_f2[:3]
    #print Z[:10],pdf_alpha_f[:3], pdf_beta_f[:3]
    
    '''
    ###################
    Calculate the ELBO
    ###################
    '''
    E_log_lambda = digamma(p_alpha_lambda) + log(p_beta_lambda)
    E_lp_lambda = ((alpha_lambda - 1) * E_log_lambda - 1.0/beta_lambda * E_lambda - log(gamma(alpha_lambda))
            - alpha_lambda * log(beta_lambda)).sum(axis=0)
    print 'lp_lam', E_lp_lambda

    # E(log(p(a|lambda)))
    E_lp_a_lambda = ( - 0.05 * tr(dot(diag(E_lambda), cov_a)).sum() - 0.5 * N *log(2.0*np.pi) 
                     + 0.05 * log(np.prod(E_lambda)))
    print 'lp_a_lam', E_lp_a_lambda

    # E(log(p(G|a,K)))
    E_g2 = arr([(diag(cov_g[i,...]) + mu_g[:,i]**2).sum() for i in range(N)])
    kga = arr([dot(K[:,:,i].T , outer(mu_g[:,i],mu_a)) for i in range(N)])
    
    kaa = arr([dot(K[:,:,i].T , dot(K[:,:,i],corr_a)) for i in range(N)])
    #print 'kaa', kaa[0,:5,:5]

    E_lp_G = np.sum([-.5 * E_g2[i] + tr(kga[i,...]) - 0.5 * tr(kaa[i,...]) 
                     - 0.5 * P *log(2.0*np.pi) for i in range(N)])
    print 'lp_g', E_lp_G

    # E(log(p(gamma)))
    E_log_gamma = digamma(p_alpha_gamma) - log(p_beta_gamma)
    print 'E_log_gamma', E_log_gamma
    E_lp_gamma = ((alpha_gamma - 1) * E_log_gamma - E_gamma/beta_gamma 
                  - log(gamma(alpha_gamma)) - alpha_gamma * log(beta_gamma))
    print 'lp_gamma', E_lp_gamma

    # E(log(p(b|gamma)))
    E_lp_b = (- 0.5 * E_gamma * E_b2 - 0.5 *log(2.0*np.pi) + 0.5 * log(E_gamma))
    print 'lp_b', E_lp_b

    # E(log(p(omega)))
    E_log_omega = digamma(p_alpha_omega) - log(p_beta_omega)
    E_lp_omega = ((alpha_omega - 1) * E_log_omega - E_omega/beta_omega - log(gamma(alpha_omega))
                  - alpha_omega * log(beta_omega) ).sum()
    print 'lp_omega', E_lp_omega

    # E(log(p(e|omega)))
    E_lp_e = ( -0.5 * tr(dot(diag(E_omega), corr_e)) - 0.5 * P * log(2.0*np.pi)
             + 0.05 * log(np.prod(E_omega)))
    print 'lp_e', E_lp_e

    # E(log(p(f|e,b,G)))
    z1 = (- 0.5 * E_f2).sum()
    z2 = ((dot(mu_b_e[1:], mu_g) + mu_b_e[0]) * mu_f ).sum()
    '''z3 = - 0.5 * arr([(tr(dot(corr_e, corr_g[i,...])) + 2.0 * dot(E_be, mu_g[:,i]) 
                       + E_b2) for i in range(N)])''' 
    
    #z3 = - 0.5 * arr([(np.sum(dot(corr_e, corr_g[i,...])) + 2.0 * dot(E_be, mu_g[:,i]) 
    #                   + E_b2) for i in range(N)]) 
    z3 = - 0.5 * np.sum(dot(corr_e, corr_G)) - 0.5 * (dot(E_be, mu_g) + E_b2).sum()
                  
    E_lp_f =  z1 + z2 + z3 - 0.5 *  N *log(2.0*np.pi)
    print 'lp_f'
    '''print mu_g[:,:3]
    print mu_b_e[1:]
    print dot(mu_b_e[1:], mu_g)[:3]
    print ((dot(mu_b_e[1:], mu_g) + mu_b_e[0]) * mu_f)[:3]
    print z1[:3]
    print z2[:3]
    print z3[:3]'''
    print E_lp_f


    # E(log(q(lambda)))
    E_lq_lambda = (-p_alpha_lambda - log(p_beta_lambda) - log(gamma(p_alpha_lambda)) 
                   - (1-p_alpha_lambda) * digamma(p_alpha_lambda)).sum()
    print 'lq_lam', E_lq_lambda

    # E(log(q(a)))
    det_cov_a = det(cov_a)
    E_lq_a = -0.5 * N * (log(2.0*np.pi)+1) - 0.5 * log(det_cov_a) if det_cov_a > 0 else 10**-6
    print 'lq_a', E_lq_a

    # E(log(q(G)))
    E_lq_G = (-0.5 * P *(log(2.0*np.pi)+1) - 0.5 * np.array([log(det(cov_g[i])) for i in range(N)])).sum()
    print 'lq_g', E_lq_G

    # E(log(q(gamma)))
    E_lq_gamma = (-p_alpha_gamma - log(p_beta_gamma) - log(gamma(p_alpha_gamma)) 
                   - (1-p_alpha_gamma) * digamma(p_alpha_gamma))
    print 'lq_gamma', E_lq_gamma

    # E(log(q(omega)))
    E_lq_omega = (-p_alpha_omega - log(p_beta_omega) - log(gamma(p_alpha_omega)) 
                   - (1-p_alpha_omega) * digamma(p_alpha_omega)).sum()
    print 'lq_omega', E_lq_omega

    # E(log(q(b,e)))
    det_cov_b_e = det(cov_b_e)
    E_lq_b_e = (-0.5 * (P+1) *(log(2.0*np.pi)+1) - 0.5 * log(det_cov_b_e) if det_cov_b_e > 0 else 10**-6 )
    print 'lq_be', E_lq_b_e

    # E(log(q(f)))
    E_lq_f = (-0.5 * (log(2.0*np.pi) + var_f) - log(Z)).sum()
    print 'lq_f', E_lq_f


    ELBO = (E_lp_lambda + E_lp_a_lambda + E_lp_G + E_lp_gamma + E_lp_b + E_lp_omega + E_lp_e + E_lp_f
            - E_lq_lambda - E_lq_a - E_lq_G - E_lq_gamma - E_lq_omega - E_lq_b_e - E_lq_f)
    
    print "ITER %d: %f" %(it, ELBO)
    
    if np.abs(ELBO - ELBO_init) < thresh:
        print 'Convergence'
        break
        
    ELBO_init = ELBO
    
    if it == MAX_ITER:
        break
        
    it += 1

    
    

Updated a...
mu_a [-0.03595476  0.14576684 -0.01581136  0.12276905 -0.01469097]
cov_a [[ 1.32625647e+00 -1.13339319e-02  1.86818222e-03  1.66515909e-02
   5.81100738e-04]
 [-1.13339319e-02  1.27990421e+00  1.10927143e-03 -3.48732054e-02
  -1.31782037e-04]
 [ 1.86818222e-03  1.10927143e-03  1.01084230e+00 -3.17885640e-03
  -1.62521373e-02]
 [ 1.66515909e-02 -3.48732054e-02 -3.17885640e-03  1.64608754e+00
  -4.31580992e-03]
 [ 5.81100739e-04 -1.31782039e-04 -1.62521373e-02 -4.31580992e-03
   1.09472684e+00]]
Updated g...
mu_g [[0.39022686 0.35376911 0.44656903]
 [0.32068976 0.58270156 0.34133815]
 [0.26835683 0.84002445 0.24858247]]
[[-0.11187788 -0.13357234 -0.06500035]
 [-0.55259546 -0.60595037 -0.44521752]
 [-0.63998485 -0.49606102 -0.5658488 ]]
cov_g [[[ 0.4 -0.1 -0.1]
  [-0.1  0.4 -0.1]
  [-0.1 -0.1  0.4]]]
corr_G [[ 99.95095493   4.42644699   4.58604558]
 [  4.42644699 125.43955948  29.64806863]
 [  4.58604558  29.64806863 138.08620219]]
Updated e:
mu_b_e [-0.01945853  0.26123435  

  [-0.12719144 -0.18319102  0.81780253]]]
corr_G [[353.93558671 206.55391541 207.39288002]
 [206.55391541 549.83295163 353.37645031]
 [207.39288002 353.37645031 562.83982509]]
Updated e:
mu_b_e [0.04829726 0.3962116  0.5932508  0.57043012]
cov_b_e [[ 0.0056297  -0.00164165  0.00059099  0.00049227]
 [-0.00164165  0.00430637 -0.00106196 -0.00099346]
 [ 0.00059099 -0.00106196  0.00330146 -0.00165049]
 [ 0.00049227 -0.00099346 -0.00165049  0.00319443]]
corr_e [[0.16129    0.23399089 0.22501757]
 [0.23399089 0.35524798 0.33675763]
 [0.22501757 0.33675763 0.32858495]]
E_be [0.01749428 0.02924338 0.02804249]
Updated f
mu_f [2.15559407 2.68673071 2.08874869] [-2.20525177 -2.18907285 -2.23566355]
var_f [0.56472077 0.78862331 0.52965681]
E_f2 [5.21130656 8.00714521 4.8925279 ]
lp_lam -203.87247763116858
lp_a_lam -193.17698799643807
lp_g -814.9703176985373
E_log_gamma 0.039864154033178695
lp_gamma -1.4949472591406363
lp_b -0.7238446948919831
lp_omega -3.9347905112243375
lp_e -3.2642871402977014
l

lp_g -815.5880321193049
E_log_gamma 0.04313823913719355
lp_gamma -1.4900606784938442
lp_b -0.7310776424376525
lp_omega -4.002997873379664
lp_e -3.1925122112855457
lp_f
-1295.0694947315803
lq_lam -195.11708267451093
lq_a -245.4379883853446
lq_g -794.6205653836246
lq_gamma -1.3543245102168495
lq_omega -3.7292510359071143
lq_be 5.955911480751988
lq_f -221.49302738837417
ITER 10: -1061.550392
Updated a...
mu_a [ 0.01429447  0.07816077 -0.01090006  0.05041936 -0.01097371]
cov_a [[ 9.35032861e-01 -7.39119880e-03  8.04296929e-04  8.02300856e-03
  -5.18307638e-04]
 [-7.39119880e-03  9.43967173e-01  1.59744388e-04 -2.00479288e-02
  -1.05937028e-03]
 [ 8.04296927e-04  1.59744390e-04  9.69165144e-01 -2.81930629e-03
  -1.95568954e-02]
 [ 8.02300856e-03 -2.00479288e-02 -2.81930629e-03  9.23475103e-01
  -3.33618582e-03]
 [-5.18307640e-04 -1.05937028e-03 -1.95568954e-02 -3.33618582e-03
   9.71887056e-01]]
Updated g...
mu_g [[1.44170017 1.7466764  1.40298481]
 [1.40037034 2.03609562 1.31081976]
 [1.32

lp_g -817.8557739104042
E_log_gamma 0.047566958530373606
lp_gamma -1.4834762090158868
lp_b -0.7399417521237608
lp_omega -4.061660482799297
lp_e -3.1363436262489066
lp_f
-1388.2192081220894
lq_lam -195.11504632609223
lq_a -245.4389998923516
lq_g -800.377584152335
lq_gamma -1.3498957908236695
lq_omega -3.772912382010507
lq_be 6.087428267006494
lq_f -230.39870785385142
ITER 14: -1142.401306
Updated a...
mu_a [ 0.01655818  0.08136337 -0.00839869  0.05513425 -0.00806504]
cov_a [[ 9.35055954e-01 -7.39332089e-03  8.04332198e-04  8.02368211e-03
  -5.18312372e-04]
 [-7.39332089e-03  9.44258164e-01  1.59762341e-04 -2.00525729e-02
  -1.05950399e-03]
 [ 8.04332198e-04  1.59762341e-04  9.69137313e-01 -2.82070164e-03
  -1.95558119e-02]
 [ 8.02368211e-03 -2.00525729e-02 -2.82070164e-03  9.23719440e-01
  -3.33740756e-03]
 [-5.18312372e-04 -1.05950399e-03 -1.95558119e-02 -3.33740756e-03
   9.71853419e-01]]
Updated g...
mu_g [[1.62815243 1.99150702 1.58148755]
 [1.57064708 2.30305124 1.4678152 ]
 [1.482

lp_g -820.5081232645871
E_log_gamma 0.05295097332485348
lp_gamma -1.4755106138423748
lp_b -0.7504407975346132
lp_omega -4.114599604505555
lp_e -3.0854864080358193
lp_f
-1466.6602596295265
lq_lam -195.11100733802448
lq_a -245.44100848671252
lq_g -805.7902408452292
lq_gamma -1.3445117760291896
lq_omega -3.812026422683119
lq_be 6.212170507352022
lq_f -237.13513272757217
ITER 19: -1211.439439
Updated a...
mu_a [ 0.01949656  0.08373065 -0.00573351  0.05964936 -0.00501656]
cov_a [[ 9.35100067e-01 -7.39570280e-03  8.04498757e-04  8.02460421e-03
  -5.18220811e-04]
 [-7.39570280e-03  9.44460037e-01  1.60391447e-04 -2.00571484e-02
  -1.05897929e-03]
 [ 8.04498758e-04  1.60391446e-04  9.69116486e-01 -2.82164809e-03
  -1.95549278e-02]
 [ 8.02460421e-03 -2.00571484e-02 -2.82164809e-03  9.23968933e-01
  -3.33827705e-03]
 [-5.18220811e-04 -1.05897929e-03 -1.95549278e-02 -3.33827705e-03
   9.71830583e-01]]
Updated g...
mu_g [[1.82478918 2.23943782 1.77116843]
 [1.7508412  2.57262979 1.63493763]
 [1.65

...

lp_g -844.7005966252841
E_log_gamma 0.12616100051139306
lp_gamma -1.3713478407628088
lp_b -0.8897435421537274
lp_omega -4.411389783720516
lp_e -2.7875169693304684
lp_f
-2364.9892445005285
lq_lam -194.67586217629395
lq_a -245.64570396287638
lq_g -839.863137240151
lq_gamma -1.2713017488426501
lq_omega -4.0231088498940615
lq_be 7.7897181821268635
lq_f -272.91899838877924
ITER 482: -2065.383254
Updated a...
mu_a [0.08302375 0.12663905 0.05264372 0.11331889 0.05632577]
cov_a [[ 9.38117263e-01 -7.42750268e-03  7.99864789e-04  8.08050364e-03
  -5.23438931e-04]
 [-7.42750267e-03  9.48633103e-01  1.79690283e-04 -2.01854084e-02
  -1.04489637e-03]
 [ 7.99864788e-04  1.79690284e-04  9.70447195e-01 -2.81969965e-03
  -1.95779336e-02]
 [ 8.08050364e-03 -2.01854084e-02 -2.81969965e-03  9.28182761e-01
  -3.33808142e-03]
 [-5.23438931e-04 -1.04489637e-03 -1.95779336e-02 -3.33808142e-03
   9.73356155e-01]]
Updated g...
mu_g [[5.7580066  6.85315347 5.64653347]
 [5.44851277 7.53156904 5.1879273 ]
 [5.15194

lp_g -844.7351825815563
E_log_gamma 0.12629355959089078
lp_gamma -1.371166068203434
lp_b -0.8899910919449117
lp_omega -4.411698591239011
lp_e -2.7871986008169767
lp_f
-2367.6990917131466
lq_lam -194.67315360043133
lq_a -245.64697469912872
lq_g -839.9020813489033
lq_gamma -1.2711691897631523
lq_omega -4.023319995014404
lq_be 7.793846817141786
lq_f -272.95755135763704
ITER 486: -2068.053093
Updated a...
mu_a [0.08321311 0.12686621 0.0528504  0.11354317 0.05653531]
cov_a [[ 9.38132063e-01 -7.42761508e-03  7.99829158e-04  8.08079360e-03
  -5.23477180e-04]
 [-7.42761508e-03  9.48659914e-01  1.79735948e-04 -2.01860829e-02
  -1.04488145e-03]
 [ 7.99829156e-04  1.79735949e-04  9.70457771e-01 -2.81971345e-03
  -1.95781309e-02]
 [ 8.08079360e-03 -2.01860829e-02 -2.81971345e-03  9.28205757e-01
  -3.33810464e-03]
 [-5.23477182e-04 -1.04488144e-03 -1.95781309e-02 -3.33810464e-03
   9.73367776e-01]]
Updated g...
mu_g [[5.77288128 6.87056876 5.66139676]
 [5.46250307 7.55032025 5.2017754 ]
 [5.1651145

lp_g -844.7693620971567
E_log_gamma 0.12642473968006301
lp_gamma -1.370986210313482
lp_b -0.890236017692382
lp_omega -4.412003574086869
lp_e -2.7868841654989844
lp_f
-2370.390251597493
lq_lam -194.67045664662191
lq_a -245.64823997715644
lq_g -839.9405506644184
lq_gamma -1.27103800967398
lq_omega -4.023528506315251
lq_be 7.797938001012373
lq_f -272.9956307970345
ITER 490: -2070.704754
Updated a...
mu_a [0.0834014  0.12709214 0.0530541  0.11376714 0.05674537]
cov_a [[ 9.38146727e-01 -7.42772643e-03  7.99793681e-04  8.08108202e-03
  -5.23515245e-04]
 [-7.42772643e-03  9.48686643e-01  1.79780883e-04 -2.01867554e-02
  -1.04486699e-03]
 [ 7.99793681e-04  1.79780884e-04  9.70468381e-01 -2.81972820e-03
  -1.95783273e-02]
 [ 8.08108203e-03 -2.01867554e-02 -2.81972820e-03  9.28228789e-01
  -3.33812829e-03]
 [-5.23515247e-04 -1.04486699e-03 -1.95783273e-02 -3.33812829e-03
   9.73379280e-01]]
Updated g...
mu_g [[5.7876596  6.88787338 5.67616513]
 [5.47640181 7.56895368 5.21553546]
 [5.17820063 8.1

lp_g -844.803143648106
E_log_gamma 0.12655455578696434
lp_gamma -1.3708082457726254
lp_b -0.8904783960177607
lp_omega -4.412304814369942
lp_e -2.7865735845593775
lp_f
-2373.063025243504
lq_lam -194.66777117432449
lq_a -245.64949983951448
lq_g -839.9785552653267
lq_gamma -1.2709081935670787
lq_omega -4.02373444062289
lq_be 7.8019923171814
lq_f -273.0332462817726
ITER 494: -2073.338528
Updated a...
mu_a [0.08358716 0.12731764 0.05325822 0.11399043 0.05695106]
cov_a [[ 9.38161298e-01 -7.42783755e-03  7.99758365e-04  8.08136907e-03
  -5.23553121e-04]
 [-7.42783755e-03  9.48713331e-01  1.79825115e-04 -2.01874258e-02
  -1.04485313e-03]
 [ 7.99758366e-04  1.79825115e-04  9.70478892e-01 -2.81974340e-03
  -1.95785223e-02]
 [ 8.08136906e-03 -2.01874258e-02 -2.81974340e-03  9.28251821e-01
  -3.33815259e-03]
 [-5.23553121e-04 -1.04485313e-03 -1.95785223e-02 -3.33815259e-03
   9.73390761e-01]]
Updated g...
mu_g [[5.80234284 6.90506882 5.6908398 ]
 [5.49021019 7.58747088 5.22920864]
 [5.19120071 8.1

lp_g -844.8365344071
E_log_gamma 0.126683043685909
lp_gamma -1.3706321248162183
lp_b -0.8907182729844751
lp_omega -4.412602383467661
lp_e -2.786266780691855
lp_f
-2375.717680182544
lq_lam -194.66509712799143
lq_a -245.65075433652856
lq_g -840.0161038871938
lq_gamma -1.270779705668134
lq_omega -4.023937847398675
lq_be 7.8060103373850485
lq_f -273.0704071183552
ITER 498: -2075.954674
Updated a...
mu_a [0.08377151 0.12754173 0.0534578  0.11421385 0.05715788]
cov_a [[ 9.38175765e-01 -7.42794786e-03  7.99723130e-04  8.08165469e-03
  -5.23590888e-04]
 [-7.42794786e-03  9.48739932e-01  1.79868599e-04 -2.01880934e-02
  -1.04483984e-03]
 [ 7.99723129e-04  1.79868599e-04  9.70489323e-01 -2.81975913e-03
  -1.95787163e-02]
 [ 8.08165469e-03 -2.01880934e-02 -2.81975913e-03  9.28274867e-01
  -3.33817758e-03]
 [-5.23590889e-04 -1.04483984e-03 -1.95787163e-02 -3.33817758e-03
   9.73402227e-01]]
Updated g...
mu_g [[5.81693231 6.92215661 5.70542216]
 [5.50392939 7.60587346 5.2427962 ]
 [5.20411586 8.141

In [47]:
corr_G

array([0., 0., 0.])

# Test

In [27]:
def predict(Y):
    def predict_one(k):
        '''
        k: PxN
        '''
        print k.shape
        mu_g = dot(k, mu_a).squeeze()
        cov_g = dot(dot(k, cov_a), k.T) + diag(np.repeat(1,P))
        
        z = concat(([1], mu_g))
        mu_f = dot(mu_b_e, z)
        var_f = dot(dot(z.T, cov_b_e),z)
        
        '''LU = arr(map(lambda x: (-10**5, -v) if x==-1 else (v,10**5), y))
        eg = dot(mu_b_e[1:].T, mu_g)
        alpha_f = LU[:,0] - eg - mu_b_e[0]
        beta_f = LU[:,1] - eg - mu_b_e[0]
        pdf_alpha_f = norm.pdf(alpha_f)
        pdf_beta_f = norm.pdf(beta_f)
        Z = norm.cdf(beta_f) - norm.cdf(alpha_f)
        mu_f = eg + mu_b_e[0] + (pdf_alpha_f - pdf_beta_f)/Z
        var_f = 1. + (alpha_f * pdf_alpha_f - beta_f * pdf_beta_f)/Z - (pdf_alpha_f - pdf_beta_f)**2/Z**2 
        E_f2 = var_f + mu_f**2'''
        
        return arr([mu_f, var_f])
    
        
    K_test = get_kernels(Y, x, poly=True)
    print K_test.shape
    
    pred = arr([predict_one(K_test[:,i,:]) for i in range(Y.shape[0])])
    
    return pred


predict(x[:5])
    

(3, 5, 200)
(3, 200)
(3, 200)
(3, 200)
(3, 200)
(3, 200)


array([[0.        , 0.00496287],
       [0.        , 0.00496287],
       [0.        , 0.00496287],
       [0.        , 0.00496287],
       [0.        , 0.00496287]])

In [296]:
rv_lambda = np.random.gamma(p_alpha_lambda, p_beta_lambda, N) 
rv_a = np.array([np.random.normal(0, sqrt(1.0/rv_lambda[i]), 1) for i in range(N)])

rv_gamma = np.random.gamma(p_alpha_gamma, p_beta_gamma, 1) 
rv_b = np.random.normal(0, sqrt(1.0/rv_gamma), 1)

rv_omega = np.random.gamma(p_alpha_omega, p_beta_omega, P) 
rv_e = np.array([np.random.normal(0, sqrt(1.0/rv_omega[i]), 1) for i in range(P)])


In [20]:
rv_a = np.random.multivariate_normal(mu_a, cov_a, 1).squeeze()
print rv_a
b_e = np.random.multivariate_normal(mu_b_e, cov_b_e, 1).squeeze()
#print b_e
rv_b, rv_e = b_e[0], b_e[1:]
print rv_b,rv_e

f = dot(rv_a.T, np.sum([rv_e[j] * K[j,...] for j in range(P)], axis=0))+ rv_b
f

[-8.15725538e-01 -1.78821961e+00  1.30275942e+00 -3.88237364e-01
 -1.31666799e+00  3.51499865e-01 -7.47951362e-02 -5.03187290e-01
  4.01008452e-01  4.60204270e-02  5.19090102e-01 -1.42428436e+00
 -3.21983231e-01  3.89329066e-01  8.61479522e-01 -2.87463423e-01
  1.65591048e-01 -6.15138762e-02 -1.46767041e+00  1.07221790e+00
 -1.32526269e-01 -1.23703615e+00  5.00405953e-01 -1.35308493e+00
 -1.03344844e+00 -1.76426808e+00  1.49423505e+00  1.49312788e+00
 -2.95696769e-01 -6.69207130e-01 -6.26190404e-01  1.48954487e+00
  1.91184555e+00 -3.30255189e-01  6.96653974e-01 -1.93871622e+00
 -7.18385060e-01  1.10527464e+00  3.20703656e-01  1.20276458e+00
  1.59547036e+00 -1.60554199e+00  1.18158917e+00  8.80496422e-01
  1.62932499e+00  6.20130262e-01 -7.02097032e-01  2.02898943e+00
 -7.30177608e-01  1.17923797e+00 -1.04785627e+00  1.10463642e-01
 -1.51436284e+00  1.18230761e+00 -1.58599688e+00  1.61325543e+00
 -2.29237746e+00  2.90555131e-01 -1.96071534e+00  3.91580257e-01
  1.83778350e+00 -1.65580

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  """Entry point for launching an IPython kernel.


array([0.07771424, 0.07636934, 0.07843961, 0.07789443, 0.07705848,
       0.07758644, 0.07708547, 0.0775206 , 0.08077685, 0.07716072,
       0.07813371, 0.07780681, 0.07759706, 0.07651381, 0.07728327,
       0.07019122, 0.07763255, 0.07721659, 0.07741297, 0.07759386,
       0.07569693, 0.07747512, 0.07734016, 0.07686826, 0.08061884,
       0.07722535, 0.0769985 , 0.07760192, 0.07878116, 0.0774381 ,
       0.07758709, 0.07670534, 0.07721442, 0.07742667, 0.07767759,
       0.07721008, 0.07691849, 0.077897  , 0.07752497, 0.076062  ,
       0.07712356, 0.07717697, 0.07708691, 0.07693342, 0.07691036,
       0.07673831, 0.07768736, 0.07762194, 0.07665162, 0.07735069,
       0.07508146, 0.08248257, 0.07742137, 0.0754015 , 0.0761512 ,
       0.07781491, 0.07754681, 0.07715656, 0.07772696, 0.07513799,
       0.07745997, 0.07740772, 0.07706324, 0.07721332, 0.07730851,
       0.07711976, 0.0775616 , 0.07721356, 0.07878876, 0.07709062,
       0.07759854, 0.07709954, 0.07730732, 0.07824008, 0.07745

In [298]:
f = dot(rv_a.T, np.sum([rv_e[j] * K[j,...] for j in range(P)], axis=0))+ rv_b
f

array([[ 1.12638506e+06, -4.61826208e+05, -4.05379471e+05,
        -1.85042912e+04, -5.16563565e+05,  4.85996868e+05,
        -5.28950160e+06,  1.46638528e+06, -9.73502635e+05,
         1.50544557e+05, -3.80025758e+06,  2.95028822e+05,
         3.21761949e+06,  6.66316482e+01,  9.22900338e+06,
        -4.07924555e+03,  1.92081602e+05, -1.77277562e+06,
         2.30534277e+05, -3.93058569e+05, -8.20253938e+05,
         9.79469487e+06, -4.66761681e+06, -2.00434846e+03,
         9.41173388e+05,  3.92750490e+04, -1.45768211e+07,
         5.80965501e+05, -5.99532381e+06, -1.80874103e+05,
        -2.68397692e+04, -5.82750348e+07, -1.48857268e+03,
         7.71777273e+05, -8.02252465e+03,  2.06170536e+04,
         2.84002645e+07,  3.59358281e+06,  1.20234811e+03,
         4.23555136e+07, -5.58964447e+05,  3.76049011e+05,
        -9.01022527e+06,  1.75189722e+05, -1.79702931e+03,
         1.26727746e+03,  1.42493107e+05,  6.46433415e+04,
        -2.78554696e+06, -1.14387765e+05,  4.59461238e+0

In [305]:
from sklearn.metrics import accuracy_score, f1_score

pred = np.sign(f).squeeze()
print 'Accuracy:', accuracy_score(y, pred)
print 'F1 score:', f1_score(y, pred)

Accuracy: 0.255
F1 score: 0.4063745019920319


In [308]:
y,f

(array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
 array([[ 1.12638506e+06, -4.61826208e+05, -4.05379471e+05,
         -1.85042912e+04, -5.16563565e+05,

In [295]:
p_alpha_lambda ,  p_beta_lambda ,  p_alpha_gamma ,  p_beta_gamma , p_alpha_omega ,    p_beta_omega 
print

In [141]:
#n[K[j][i,:] for j in range(P)]
print mu_g.shape
#np.sum((dot(K[i,:,:], mu_g[i,:]) for i in range(P))).shape
#K[:,0,:].shape
#z.shape,z[:,1].shape,  (mu_f[1]* mu_e).shape,  E_be.shape
#arr([(diag(cov_g[i,...]) + mu_g[:,i]).sum() for i in range(N)]).shape
#dot(K[:,:,0].T , outer(mu_g[:,0],mu_a)).shape

#[dot(K[:,:,i].T , outer(mu_g[:,i],mu_a)) for i in range(N)]
#(cov_g + arr([outer(mu_g[:,i], mu_g[:,i]) for i in range(N)])).shape

from scipy.stats import norm

v = 1
LU = arr(map(lambda x: (-np.inf, -v) if x==-1 else (v,np.inf), y))
eg = dot(mu_e.T, mu_g)
alpha_f = LU[:,0] - eg - mu_b
beta_f = LU[:,1] - eg - mu_b
pdf_alpha_f = norm.pdf(alpha_f)
pdf_beta_f = norm.pdf(beta_f)
Z = norm.cdf(beta_f) - norm.cdf(alpha_f)
mu_f = eg + mu_b + (pdf_alpha_f - pdf_beta_f)/Z
var_f = 1. + (alpha_f * pdf_alpha_f - beta_f * pdf_beta_f)/Z - (pdf_alpha_f - pdf_beta_f)**2/Z**2 

(3, 200)


/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)
/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python2.7/dist-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


In [137]:
dot(mu_e.T, mu_g) - mu_b
from scipy.stats import norm
norm.cdf([np.inf,2])

array([1.        , 0.97724987])

In [147]:
E_g2 = arr([(diag(cov_g[i,...]) + mu_g[:,i]**2).sum() for i in range(N)])
#print 'eg2',E_g2.shape
kga = arr([dot(K[:,:,i].T , outer(mu_g[:,i],mu_a)) for i in range(N)])
#print 'kga', kga.shape

#print 'corra', corr_a.shape
kaa = arr([dot(K[:,:,i].T , dot(K[:,:,i],corr_a)) for i in range(N)])
#print 'kaa', kaa.shape

E_lp_G = np.sum([-.5 * E_g2[i] + tr(kga[i,...]) - 0.5 * tr(kaa[i,...]) 
                 - 0.5 * P *log(2.0*np.pi) for i in range(N)])
print 'lp_g', E_lp_G

ValueError: all the input arrays must have same number of dimensions

In [188]:
z = np.sum([dot(K[i,:,:], mu_g[i,:]) for i in range(P)], axis=0)
print z
mu_a = dot(cov_a, z)
print mu_a
cov_a = diag(E_lambda) + K2

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan 

In [208]:
inv = np.linalg.inv

cov_b_e = np.zeros((P+1,P+1))
cov_b_e[0,0] = var_b
cov_b_e[1:,1:] = cov_e
print cov_b_e
z = dot(cov_b_e[0,1:],inv(cov_e))
print z[:3]
E_be = mu_b * mu_e + dot(cov_e, z)
print 'E_be', E_be

z = dot(K, mu_a)
print z[:5,:5]
mu_g = cstack([dot(cov_g[i,:,:], (z[:,i] + mu_f[i]* mu_e + E_be)) for i in range(N)])
mu_g

[[0.60913865 0.         0.         0.        ]
 [0.         0.46786111 0.         0.        ]
 [0.         0.         0.6651148  0.        ]
 [0.         0.         0.         0.59561409]]
[0. 0. 0.]
E_be [0. 0. 0.]
[[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]


array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan

In [225]:
v = 1
LU = arr(map(lambda x: (-10**5, -v) if x==-1 else (v,10**5), y))
print LU[:5]
eg = dot(mu_e.T, mu_g)
alpha_f = LU[:,0] - eg - mu_b
beta_f = LU[:,1] - eg - mu_b
print alpha_f[:4],beta_f[:4]
pdf_alpha_f = norm.pdf(alpha_f)
pdf_beta_f = norm.pdf(beta_f)
print pdf_alpha_f[:4],pdf_beta_f[:4]
Z = norm.cdf(beta_f) - norm.cdf(alpha_f)
print Z[:4]
mu_f = eg + mu_b + (pdf_alpha_f - pdf_beta_f)/Z
var_f = 1. + (alpha_f * pdf_alpha_f - beta_f * pdf_beta_f)/Z - (pdf_alpha_f - pdf_beta_f)**2/Z**2 
print var_f[:5]
E_f2 = var_f + mu_f**2

E_lp_f = (-0.5 * E_f2 + (dot(mu_e.T,mu_g) + mu_b) * mu_f 
              - 0.5 * arr([(tr(dot(corr_e,corr_g[i,...])) + 2 * dot(E_be.T, mu_g[:,i]) + E_b2) 
                           for i in range(N)]) 
              - 0.5* log(2.0*np.pi)).sum()

[[     1 100000]
 [     1 100000]
 [     1 100000]
 [     1 100000]
 [     1 100000]]
[1. 1. 1. 1.] [100000. 100000. 100000. 100000.]
[0.24197072 0.24197072 0.24197072 0.24197072] [0. 0. 0. 0.]
[0.15865525 0.15865525 0.15865525 0.15865525]
[0.19909767 0.19909767 0.19909767 0.19909767 0.19909767]


In [267]:
#z = dot(cov_b_e[0,1:],inv(cov_e))
#E_be = mu_b * mu_e + dot(cov_e, z)
print 'E_be', E_be
#z = dot(K, mu_a)
#print 'Ka', z[:5,:5]
print 'cov_g', cov_g[:5,:5]
mu_g = cstack([dot(cov_g[i,:,:], (dot(K[:,i,:], mu_a) + mu_f[i]* mu_e + E_be)) for i in range(N)])

array([[-3.21174770e-89, -9.51330485e-50,  7.41605516e-53,
        -1.46269847e-56],
       [-9.51330485e-50,  1.08494586e-13,  2.58343028e-15,
        -1.03037943e-18],
       [ 7.41605516e-53,  2.58343028e-15,  6.15156408e-17,
        -2.45352309e-20],
       [-1.46269847e-56, -1.03037943e-18, -2.45352309e-20,
         9.78886525e-24]])

In [260]:
log(det(inv(diag(E_lambda) + K2))) if det(inv(diag(E_lambda) + K2))!=0 else 10**-6# + np.random.normal(scale=10**-3,size=cov_a.shape))


1e-06

In [281]:
mu_a

array([ 3.94455581e+47,  1.27025937e+49, -6.57706512e+49,  7.95515751e+49,
        2.21548718e+49,  4.07051042e+48, -1.03261371e+48, -3.41738849e+48,
       -2.89784803e+48, -1.66023250e+49,  1.05188155e+48, -6.94712814e+48,
       -8.86142411e+47,  4.21576855e+49,  3.00168030e+47, -2.65004215e+48,
       -2.13687525e+49,  1.12230730e+48, -8.03866042e+48,  4.56567617e+48,
       -1.15294853e+48, -5.50694602e+47,  1.19540915e+48, -6.67059890e+48,
       -1.98530154e+48,  9.35521613e+49,  2.59759080e+47,  3.72636532e+48,
        1.63277136e+48,  1.10361036e+49, -1.47028813e+48, -1.40583898e+47,
       -7.24310363e+48,  1.57318377e+48, -3.16550158e+48, -3.23012914e+48,
        9.70528431e+46, -8.01845594e+47, -1.51368538e+48,  1.07400389e+47,
        5.85555127e+48, -5.87919006e+48, -1.03921473e+46,  7.32311871e+48,
       -3.87479908e+48, -5.27439385e+48, -4.31858391e+48, -4.43429802e+49,
        8.21202641e+47, -6.44562899e+48, -3.41754460e+48,  3.23271602e+47,
        4.51875540e+48, -

In [207]:
cov_b_e = np.zeros((P+1,P+1))
cov_b_e[0,0] = var_b
cov_b_e[1:,1:] = cov_e
cov_b_e

array([[0.60913865, 0.        , 0.        , 0.        ],
       [0.        , 0.46786111, 0.        , 0.        ],
       [0.        , 0.        , 0.6651148 , 0.        ],
       [0.        , 0.        , 0.        , 0.59561409]])

In [257]:
E_log_gamma = digamma(p_alpha_gamma) - log(p_beta_gamma)
    E_lp_gamma = ((alpha_gamma - 1) * E_log_gamma - E_gamma/beta_gamma 
                  - log(gamma(alpha_gamma)) - alpha_gamma * log(beta_gamma))
    print 'lp_gamma', E_lp_gamma

198

In [12]:
E_log_gamma

array([1.9818309])

In [280]:
mu_g[:,1]

array([-8.60894603e+47, -1.12326416e+48, -1.11950883e+48])

In [284]:
t = np.zeros((3,2,2))
t[:,:,:] = np.eye(2)
print t

[[[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]]


In [31]:
hh = arr(np.zeros((2,5)))
print hh.shape
hh[2,:] = arr(np.repeat(1,5))
hh

(2, 5)


IndexError: index 2 is out of bounds for axis 0 with size 2

In [20]:
E_lq_gamma = (-p_alpha_gamma - log(p_beta_gamma) - log(gamma(p_alpha_gamma)) 
                   - (1-p_alpha_gamma) * digamma(p_alpha_gamma))
print 'lq_gamma', E_lq_gamma

lq_gamma -1.0


In [19]:
(-p_alpha_gamma - log(p_beta_gamma) - log(gamma(p_alpha_gamma)) 
                   - (1-p_alpha_gamma) * digamma(p_alpha_gamma))

-1.0

In [37]:
get_ploynomial_kernel(x)

array([[4.54806738e+00, 1.55250475e-01, 3.54520992e+00, ...,
        4.47166480e+02, 4.04098730e+02, 4.00066206e+02],
       [1.55250475e-01, 2.64806807e+00, 6.51851854e-02, ...,
        1.70100749e+01, 1.64479107e+01, 1.62326095e+01],
       [3.54520992e+00, 6.51851854e-02, 3.73608056e+00, ...,
        1.35097231e+02, 1.26550280e+02, 1.25338946e+02],
       ...,
       [4.47166480e+02, 1.70100749e+01, 1.35097231e+02, ...,
        2.16082229e+05, 1.91086542e+05, 1.89085003e+05],
       [4.04098730e+02, 1.64479107e+01, 1.26550280e+02, ...,
        1.91086542e+05, 1.69066421e+05, 1.67295780e+05],
       [4.00066206e+02, 1.62326095e+01, 1.25338946e+02, ...,
        1.89085003e+05, 1.67295780e+05, 1.65543706e+05]])

In [60]:
dot(mu_b_e[1:], mu_g).shape

(200,)

In [82]:
y

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])